In [0]:
#Installing spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
#setting environment
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7.tgz"

In [0]:
#building session
import findspark
findspark.init('/content/spark-2.4.5-bin-hadoop2.7')
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
#mounting drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/Colab\ Notebooks

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks


In [0]:
#Downloading products ratings data
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Clothing_Shoes_and_Jewelry.csv

--2020-05-04 04:57:25--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Clothing_Shoes_and_Jewelry.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 992604625 (947M) [application/octet-stream]
Saving to: ‘Clothing_Shoes_and_Jewelry.csv’

Clothing_Shoes_and_ 100%[===================>] 946.62M  10.8MB/s    in 89s     

2020-05-04 04:58:54 (10.7 MB/s) - ‘Clothing_Shoes_and_Jewelry.csv’ saved [992604625/992604625]



In [0]:
#Downloading metadata for the products
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Clothing_Shoes_and_Jewelry.json.gz

--2020-05-04 05:00:02--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Clothing_Shoes_and_Jewelry.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1519327531 (1.4G) [application/octet-stream]
Saving to: ‘meta_Clothing_Shoes_and_Jewelry.json.gz’

meta_Clothing_Shoes 100%[===================>]   1.41G  10.8MB/s    in 2m 14s  

2020-05-04 05:02:17 (10.8 MB/s) - ‘meta_Clothing_Shoes_and_Jewelry.json.gz’ saved [1519327531/1519327531]



## **Data Pre-Processing**

In [0]:
#importing libraries
import pandas as pd
import gzip
import json

In [0]:
#Coverting the metadata file from json to DataFrame
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

Beauty_metaData = getDF('meta_Clothing_Shoes_and_Jewelry.json.gz')

In [10]:
#looking at the metadata
Beauty_metaData.head()

,category,description,title,brand,feature,rank,date,asin,image,also_view,price,fit,also_buy,main_cat,tech1,details,similar_item,tech2
0,"[Clothing, Shoes & Jewelry, Costumes & Accesso...","[6"" long, stretched waist measures 11 1/2"" acr...",Purple Sequin Tiny Dancer Tutu Ballet Dance Fa...,Big Dreams,"[3 layers of tulle, 6"" long, stretched waist m...","19,963,069inClothing,ShoesJewelry(",5 star5 star (0%),0000037214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[Clothing, Shoes & Jewelry, Luggage & Travel G...",[The Hottest Bag in Town! Brand: Anello Condit...,Japan Anello Backpack Unisex LARGE LIGHT GRAY ...,Anello,"[Polyester Canvas Waterproof, Imported, Size: ...","4,537,420inClothing,ShoesJewelry(",5 star,0201377179,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[Clothing, Shoes & Jewelry, Luggage & Travel G...",[The Hottest Bag in Town! Brand: Anello Condit...,Japan Anello Backpack Unisex PINK BEIGE LARGE ...,Anello,"[PU Leather, Imported, Size: Large PU Leather ...","3,994,472inClothing,ShoesJewelry(",5 star,0204444454,[https://images-na.ssl-images-amazon.com/image...,"[B07CJF5HLR, B01EBI8UH6, B01N1LR908, B074M4QKG...",$70.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[Clothing, Shoes & Jewelry, Luggage & Travel G...",[The Hottest Bag in Town! Brand: Anello Condit...,Japan Anello Backpack Unisex BLACK LARGE PU LE...,Anello,"[PU Leather, Imported, Size: Large PU Leather ...","635,761inClothing,ShoesJewelry(",5 star,0204444403,[https://images-na.ssl-images-amazon.com/image...,"[B01DLVYOPG, B07CJF5HLR, B01MSS59Y6, B071YBC1J...",$65.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[Clothing, Shoes & Jewelry, Novelty & More, Cl...",[Brand New. Hat Centre Length: adult about 8c...,bettyhome Unisex Adult Winter Spring Thicken C...,bettyhome,"[Imported, Plush,Cotton, Polyester, Lovely for...","5,061,041inClothing,ShoesJewelry(",5 star5 star (0%),0206313535,[https://images-na.ssl-images-amazon.com/image...,"[B00VGT93YS, B07FTY87LM, B01IKVOY8E, B01GF6C6F...",$18.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
#Checking null values in the columns
Beauty_metaData.isna().sum()

category              0
description      671407
title                50
brand            802651
feature          162958
rank              69814
date             185208
asin                  0
image            684447
also_view       1943585
price           1784162
fit             2291215
also_buy        2225495
main_cat        2471831
tech1           2676368
details         2570076
similar_item    2667932
tech2           2684995
dtype: int64

In [0]:
#Choosing only asin, title and main_cat as they have the least missing values and are useful for our later analysis.
Beauty_metaData1 = Beauty_metaData[["asin", "title", "main_cat"]]

In [19]:
#Checking the data again
Beauty_metaData1.head(40)

,asin,title,main_cat
0,0000037214,Purple Sequin Tiny Dancer Tutu Ballet Dance Fa...,NaN
1,0201377179,Japan Anello Backpack Unisex LARGE LIGHT GRAY ...,NaN
2,0204444454,Japan Anello Backpack Unisex PINK BEIGE LARGE ...,NaN
3,0204444403,Japan Anello Backpack Unisex BLACK LARGE PU LE...,NaN
4,0206313535,bettyhome Unisex Adult Winter Spring Thicken C...,NaN
5,0206335962,bettyhome Womens Lace Short Sleeves Top Printi...,NaN
6,0206323190,bettyhome Women Sexy Club Party Super High Hee...,NaN
7,0244444404,Japan Anello Backpack Unisex NAVY MINI SMALL P...,NaN
8,024444448X,Japan Anello LARGE CAMO 2 Way Unisex Shoulder ...,NaN
9,0681256036,Twilight Saga: New Moon Alice Chocker Necklace...,Toys & Games


In [20]:
#dropping all the rows with missing values from meta data
Products_meta_df = Beauty_metaData1.dropna()
display(Products_meta_df)

,asin,title,main_cat
9,0681256036,Twilight Saga: New Moon Alice Chocker Necklace...,Toys & Games
10,0756029104,Viva Español Pins Set of 10,Office Products
11,0829750487,Todo lo puedo / I Can Do Everything Bible Cove...,Books
12,0871167042,Organic Wire and Metal Jewelry: Stunning Piece...,Books
13,096603550X,Coach Soccer:Everything You Need to VHS,Movies & TV
...,...,...,...
2684970,B01HJE127E,Columbia Junior Mesh Ball Cap,Sports & Outdoors
2684978,B01HJDZM30,Columbia PFG Mesh Snap Back Ball Cap,Sports & Outdoors
2685010,B01HJFWM36,Irises Vincent Van Gogh Foldable Umbrella 8 ribs,Amazon Home
2685045,B01HJFOSMO,JXK Women's Snoopy Logo T-shirt,Books


In [0]:
#renaming the columns
Products_meta_df.columns = ["itemID", "title", "category"]
display(Products_meta_df)

,itemID,title,category
9,0681256036,Twilight Saga: New Moon Alice Chocker Necklace...,Toys & Games
10,0756029104,Viva Español Pins Set of 10,Office Products
11,0829750487,Todo lo puedo / I Can Do Everything Bible Cove...,Books
12,0871167042,Organic Wire and Metal Jewelry: Stunning Piece...,Books
13,096603550X,Coach Soccer:Everything You Need to VHS,Movies & TV
...,...,...,...
2684970,B01HJE127E,Columbia Junior Mesh Ball Cap,Sports & Outdoors
2684978,B01HJDZM30,Columbia PFG Mesh Snap Back Ball Cap,Sports & Outdoors
2685010,B01HJFWM36,Irises Vincent Van Gogh Foldable Umbrella 8 ribs,Amazon Home
2685045,B01HJFOSMO,JXK Women's Snoopy Logo T-shirt,Books


In [0]:
#reading the ratings csv
Products_df = pd.read_csv("Clothing_Shoes_and_Jewelry.csv", header=None, names=["itemID", "userID", "rating"])

In [0]:
#checking the format
display(Products_df)

,itemID,userID,rating
0,0871167042,A2IC3NZN488KWK,5.0
1,0871167042,A3OT9BYASFGU2X,4.0
2,0871167042,A28GK1G2KDXHRP,5.0
3,0871167042,A3NFXFEKW8OK0E,5.0
4,0871167042,A3I6G5TKBVJEK9,5.0
...,...,...,...
32292094,B01HJHMQW6,A2ATY4H4A3RK05,5.0
32292095,B01HJHA4W0,ACNWGCHFOPZ3N,3.0
32292096,B01HJI0G5Y,A1R7H90HCU8WJK,3.0
32292097,B01HJI0G5Y,A3VZ29X7SM3L09,5.0


In [0]:
#Reducing the rows to 100,000
Products_df = Products_df.sample(n=100000, random_state=666)


In [0]:
#Importing library for encoding user_id and item_id
from sklearn import preprocessing as prep

In [0]:
#Encoding item_id in metadata
item_encoder = prep.LabelEncoder()
item_encoder.fit(Products_meta_df["itemID"])
encoded_items = item_encoder.transform(Products_meta_df["itemID"])


In [0]:
#Encoding user_id in ratings file
user_encoder = prep.LabelEncoder()
user_encoder.fit(Products_df["userID"])
encoded_users = user_encoder.transform(Products_df["userID"])

In [0]:
#Storing item_encoding as a dictionary
item_dict = {label:index for index, label in enumerate(item_encoder.classes_.tolist())}

count = len(item_dict)
Products_df["item_index"] = 0

unknown = []

for i in range(len(Products_df)):
  try:
    Products_df.iloc[i, 3] = item_dict[Products_df.iloc[i, 0]]
  except KeyError:
    count += 1
    unknown.append(Products_df.iloc[i, 0])
    item_dict[Products_df.iloc[i, 0]] = count
    Products_df.iloc[i, 3] = item_dict[Products_df.iloc[i, 0]]

In [0]:
#Creating dataframe consisting of unknown values to be merged with metadata
new_rows = [[itemID, "Unknown Item - " + str(itemID), "Others", item_dict[itemID]] for itemID in unknown if len(unknown) > 0 ]

if len(new_rows) > 0:
  new_rows = pd.DataFrame(data = new_rows, columns = ["itemID", "title", "category", "item_index"])

In [0]:
#Joining item_index to metadata as a column
encoded_items = pd.DataFrame(data = encoded_items.tolist(), columns=["item_index"])
Products_meta_df = pd.concat([Products_meta_df.reset_index(drop=True), encoded_items.reset_index(drop=True)], axis=1)

#Appending unknown items to the metadata
if len(unknown) > 0:
   Products_meta_df = Products_meta_df.append(new_rows)

In [0]:
#checking the metadata after processing
Products_meta_df

,itemID,title,category,item_index
0,0681256036,Twilight Saga: New Moon Alice Chocker Necklace...,Toys & Games,0
1,0756029104,Viva Español Pins Set of 10,Office Products,1
2,0829750487,Todo lo puedo / I Can Do Everything Bible Cove...,Books,2
3,0871167042,Organic Wire and Metal Jewelry: Stunning Piece...,Books,3
4,096603550X,Coach Soccer:Everything You Need to VHS,Movies & TV,4
...,...,...,...,...
65482,B00NNSJD0O,Unknown Item - B00NNSJD0O,Others,278203
65483,B0154LAPY0,Unknown Item - B0154LAPY0,Others,278204
65484,B00V9JYGN8,Unknown Item - B00V9JYGN8,Others,278205
65485,B00FYIXFCS,Unknown Item - B00FYIXFCS,Others,278206


In [0]:
#Joining user_index to products_df as a column
encoded_users = pd.DataFrame(data = encoded_users.tolist(), columns=["user_index"])
Products_df = pd.concat([Products_df.reset_index(drop=True), encoded_users.reset_index(drop=True)], axis=1)

#Dropping unnecessary columns
Products_df = Products_df.drop(columns = ["itemID", "userID"])

In [0]:
#Exporting the product_df and metadata_Df to csv
Products_df.to_csv("Products_preprocessed.csv", sep = ",", index=False)
Products_meta_df.to_csv("Products_meta_preprocessed.csv", sep = ",", index=False)

In [0]:
#Storing item_encoding as a dictionary
user_dict = {label:index for index, label in enumerate(user_encoder.classes_.tolist())}

#Storing it as pickled/serialized object
import pickle
with open('user_dict.pkl', 'wb') as f:
  pickle.dump(user_dict, f, pickle.HIGHEST_PROTOCOL)

# **Model Building**

In [0]:
#importing necessary libraries for ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [0]:
#setting parameter grid for parameter tuning by cross-validation
evaluator = RegressionEvaluator(metricName = "rmse", labelCol = "rating",
                                predictionCol = "prediction")
#Model
recsys = ALS(userCol = "user_index", itemCol = "item_index",
             ratingCol = "rating", nonnegative = True, coldStartStrategy="drop")

#Parameter grid
paramGrid = ParamGridBuilder() \
    .addGrid(recsys.regParam, [0.1, 0.01, 0.001]) \
    .addGrid(recsys.rank, [5, 10, 15]) \
    .build()

#Cross validation 
cvs = CrossValidator(estimator = recsys,
                     estimatorParamMaps = paramGrid,
                     evaluator = evaluator,
                     numFolds=5)

In [0]:
#Reading the Products ratings file
pathToFile = 'Products_preprocessed.csv'
ProductsDF = spark.read.csv(pathToFile, inferSchema = True, header = True)
ProductsDF.printSchema()

root
 |-- rating: double (nullable = true)
 |-- item_index: integer (nullable = true)
 |-- user_index: integer (nullable = true)



In [0]:
#Reading the metadata file
pathToFile = 'Products_meta_preprocessed.csv'
metadataDF = spark.read.csv(pathToFile, inferSchema = True, header = True)
metadataDF.printSchema()

root
 |-- itemID: string (nullable = true)
 |-- title: string (nullable = true)
 |-- category: string (nullable = true)
 |-- item_index: string (nullable = true)



In [0]:
#cross validation on the products ratings data
cvs_model = cvs.fit(ProductsDF)

#defining best model 
recsys = cvs_model.bestModel

In [0]:
#Getting top 10 recommendations for all users
userRecs = recsys.recommendForAllUsers(10)
userRecs.show()

+----------+--------------------+
|user_index|     recommendations|
+----------+--------------------+
|       148|[[248744, 7.09437...|
|       463|[[234408, 6.44202...|
|       471|[[258136, 7.60827...|
|       496|[[270400, 7.04140...|
|       833|[[251473, 7.57953...|
|      1088|[[222202, 6.49615...|
|      1238|[[236750, 7.92593...|
|      1342|[[239566, 7.15092...|
|      1580|[[263520, 5.08429...|
|      1591|[[263720, 4.99947...|
|      1645|[[47875, 8.959118...|
|      1829|[[256197, 7.90000...|
|      1959|[[239488, 5.89850...|
|      2122|[[249748, 5.61986...|
|      2142|[[237956, 5.04347...|
|      2366|[[227411, 6.67051...|
|      2659|[[275262, 5.25661...|
|      2866|[[225141, 9.16360...|
|      3175|[[224268, 3.35245...|
|      3749|[[243042, 6.04326...|
+----------+--------------------+
only showing top 20 rows



In [0]:
#printing schema for the recommendation engine
userRecs.printSchema()

root
 |-- user_index: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- item_index: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



In [0]:
#itertaing list of lists for recommedations
df = userRecs.toPandas()
list = df.values.tolist()
print(list[3])


[496, [Row(item_index=270400, rating=7.041402816772461), Row(item_index=247712, rating=6.570556640625), Row(item_index=257698, rating=6.507608413696289), Row(item_index=261606, rating=6.4802021980285645), Row(item_index=246604, rating=6.463449478149414), Row(item_index=265610, rating=6.421609878540039), Row(item_index=245889, rating=6.409777641296387), Row(item_index=276399, rating=6.393669128417969), Row(item_index=48441, rating=6.388440132141113), Row(item_index=226587, rating=6.331538200378418)]]


In [0]:
#Converting list to dataframe
import numpy as np
new =[]
dfi = pd.DataFrame(columns = ['a','b','c'])
for i in list:
  for j in i[1]:
    #new= [int(x) for x in str(i[0])]
    new=[i[0]]
    #print(type(new))
    new.append(j[0])
    new.append(j[1])
    dfi = dfi.append({'a': new[0], 'b' : new[1], 'c' : new[2]} , ignore_index = True)


In [0]:
#checking the dataframe
dfi

,a,b,c
0,148.0,248744.0,7.094375
1,148.0,241259.0,6.965418
2,148.0,216979.0,6.932223
3,148.0,214536.0,6.915580
4,148.0,225582.0,6.893297
...,...,...,...
988345,98417.0,224032.0,5.303926
988346,98417.0,187214.0,5.289124
988347,98417.0,213229.0,5.264712
988348,98417.0,251776.0,5.264226


In [0]:
#changing the datatype of columns
dfi[['b']] = dfi[['b']].astype('str')


,a,b,c
0,148.0,248744,7.094375
1,148.0,241259,6.965418
2,148.0,216979,6.932223
3,148.0,214536,6.915580
4,148.0,225582,6.893297
...,...,...,...
988345,98417.0,224032,5.303926
988346,98417.0,187214,5.289124
988347,98417.0,213229,5.264712
988348,98417.0,251776,5.264226


In [0]:
#joining the dataframes metadata and ratings data to get the categories if recommended products 
join = pd.merge(dfi, metadataDF_panda, left_on = 'b', right_on = 'item_index', how = 'left', left_index = False, right_index = False )

In [0]:
#renaming columns
join = join.rename(columns={'a': 'user_id', 'c': 'rating'})
join

,user_id,rating,itemID,title,category,item_index
0,148.0,7.094375,B00GJCX4AG,Unknown Item - B00GJCX4AG,Others,248744
1,148.0,6.965418,B01FJXWR80,Unknown Item - B01FJXWR80,Others,241259
2,148.0,6.932223,B000WAN2KS,Unknown Item - B000WAN2KS,Others,216979
3,148.0,6.915580,B0044PP6LK,Unknown Item - B0044PP6LK,Others,214536
4,148.0,6.893297,B004XYJZ7I,Unknown Item - B004XYJZ7I,Others,225582
...,...,...,...,...,...,...
988850,98417.0,5.303926,B005LHG8IA,Unknown Item - B005LHG8IA,Others,224032
988851,98417.0,5.289124,B018J5UAWK,Costa Redfish Species LS Shirts,Sports & Outdoors,187214
988852,98417.0,5.264712,B00KQ7MIRK,Unknown Item - B00KQ7MIRK,Others,213229
988853,98417.0,5.264226,B00E7NOTCG,Unknown Item - B00E7NOTCG,Others,251776


In [0]:
#converting the dataframe to csv
join.to_csv('joined.csv')